<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [29]:
import collections
from datetime import datetime, date
from pymongo import MongoClient
from bson.objectid import ObjectId

In [22]:
!docker run --name NARCIS -v /Users/dirk/local/narcis/MongoDB-NARCIS-Dec17:/data/db -p 27019:27017 -d mongo --logpath /data/db/log.log

52e2690072c9348416c76ab09f94369dada8cba0a8282d9309193d5bcf328288


In [20]:
!docker stop NARCIS

NARCIS


In [21]:
!docker rm NARCIS

NARCIS


In [30]:
client = MongoClient('mongodb://localhost:27019/')

In [31]:
client.database_names()

['NARCIS', 'admin', 'config', 'local']

In [32]:
DBN = client.NARCIS

In [33]:
DBN.collection_names()

['Dec2017']

In [34]:
DBND = DBN.Dec2017

In [35]:
DBND.count()

1622397

In [42]:
nonEmptyFields = collections.Counter()
emptyFields = collections.Counter()

i = 0
j = 0
chunk = 10000

for doc in DBND.find():
    j += 1
    i += 1
    if j == chunk:
        j = 0
        print(f'\t{i:>7}')
    for (key, val) in doc.items():
        if val:
            nonEmptyFields[key] += 1
        else:
            emptyFields[key] += 1
print(f'\t{i:>7}')

	  10000
	  20000
	  30000
	  40000
	  50000
	  60000
	  70000
	  80000
	  90000
	 100000
	 110000
	 120000
	 130000
	 140000
	 150000
	 160000
	 170000
	 180000
	 190000
	 200000
	 210000
	 220000
	 230000
	 240000
	 250000
	 260000
	 270000
	 280000
	 290000
	 300000
	 310000
	 320000
	 330000
	 340000
	 350000
	 360000
	 370000
	 380000
	 390000
	 400000
	 410000
	 420000
	 430000
	 440000
	 450000
	 460000
	 470000
	 480000
	 490000
	 500000
	 510000
	 520000
	 530000
	 540000
	 550000
	 560000
	 570000
	 580000
	 590000
	 600000
	 610000
	 620000
	 630000
	 640000
	 650000
	 660000
	 670000
	 680000
	 690000
	 700000
	 710000
	 720000
	 730000
	 740000
	 750000
	 760000
	 770000
	 780000
	 790000
	 800000
	 810000
	 820000
	 830000
	 840000
	 850000
	 860000
	 870000
	 880000
	 890000
	 900000
	 910000
	 920000
	 930000
	 940000
	 950000
	 960000
	 970000
	 980000
	 990000
	1000000
	1010000
	1020000
	1030000
	1040000
	1050000
	1060000
	1070000
	1080000
	1090000
	1100000
	1110000
	

In [43]:
len(nonEmptyFields)

15

In [44]:
len(emptyFields)

0

In [46]:
for (field, amount) in sorted(nonEmptyFields.items(), key=lambda x: (-x[1], x[0])):
    print(f'{field:<20} {amount:>7}x')

DAI                  1622397x
GlobalIDs            1622397x
ID                   1622397x
Journal              1622397x
Keywords             1622397x
NumberofIDs          1622397x
_id                  1622397x
access               1622397x
date_harv            1622397x
date_header          1622397x
date_orig            1622397x
filenr               1622397x
nldidlnorm           1622397x
originURL            1622397x
setSpec              1622397x
